In [38]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [11]:
# make request to wikepdia page
res = requests.get('https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore')
# parse the page
soup = BeautifulSoup(res.text, 'html.parser')

td_elements = soup.find_all('td')

In [24]:
td_elements[7]

<td class="navbox-list-with-group navbox-list navbox-even hlist" style="width:100%;padding:0"><div style="padding:0 0.25em">
<ul><li>iMall (2024)</li>
<li>Pasir Ris Mall (2025)</li></ul>
</div></td>

In [36]:
mall_names = []

for element in td_elements[2:7]:
    # Parse the HTML
    soup = BeautifulSoup(str(element), "html.parser")

    # Find all list items
    more_mall_names = [item.get_text() for item in soup.find_all('li')]

    mall_names += more_mall_names

print(mall_names)

['100 AM', '600 @ Toa Payoh', 'Anchorpoint', 'Beauty World Centre', 'Beauty World Plaza', 'Causeway Point', 'Century Square', 'Change Alley', 'City Square Mall', 'City Vibe', 'CityLink Mall', 'Eastpoint Mall', 'ERA APAC Centre', 'FairPrice Hub', 'Forum The Shopping Mall', 'Fu Lu Shou Complex', 'Holland Road Shopping Centre', 'Holland Piazza', 'Hougang 1', 'Hougang Mall', 'IMM', 'Jem', 'KINEX', 'Leisure Park Kallang', 'Lot One', 'Marina Bay Link Mall', 'Mustafa Centre', 'myVillage @ Serangoon', 'Nex', 'Orchard Central', 'Plaza Singapura', 'Queensway Shopping Centre', 'The Rail Mall', 'Rivervale Mall', 'Rochester Mall', 'Seletar Mall', 'Sembawang Shopping Centre', 'Sim Lim Square', 'Sim Lim Tower', 'The Centrepoint', 'The Majestic', 'The Star Vista', 'Tampines 1', 'Tampines Mall', 'Tekka Centre', 'Thomson Plaza', 'Upper Serangoon Shopping Centre', 'VivoCity', 'West Mall', 'White Sands', 'Wisma Geylang Serai', 'Yew Tee Point', '313@Somerset', 'AMK Hub', 'Alexandra Retail Centre', 'Bedok M

In [39]:
import requests

def get_geocoordinates_from_address(address: str) -> tuple:
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    
    response = requests.get(url)    
    data = response.json()

    # Check if there are results
    if data['results']:
        first_result = data['results'][0]
        latitude = first_result['LATITUDE']
        longitude = first_result['LONGITUDE']
        return (latitude, longitude)
    else:
        return ("NA", "NA")

In [40]:
latitude_list = []
longitude_list = []

count = 1
for mall in mall_names:
    try:
        geocoordinates = get_geocoordinates_from_address(mall)
        print(count, geocoordinates)
    except ConnectionError:
        geocoordinates = ("TBC", "TBC")
   
    latitude, longitude = geocoordinates
    
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    count += 1

1 ('1.27458821795426', '103.84347073661')
2 ('1.33403623894465', '103.850977706475')
3 ('1.28861469030586', '103.805009964592')
4 ('1.34241264188642', '103.776539385406')
5 ('1.34180018619223', '103.776259359854')
6 ('1.43590007901468', '103.785795816344')
7 ('1.35236829566862', '103.943812302253')
8 ('NA', 'NA')
9 ('1.31142103107683', '103.856624019991')
10 ('NA', 'NA')
11 ('1.2927777312893', '103.854173501417')
12 ('1.34270592355576', '103.953024618234')
13 ('1.33167653374942', '103.849383945185')
14 ('1.32561767765205', '103.678409741897')
15 ('1.30655430501572', '103.828652459744')
16 ('1.30151258571919', '103.854497235401')
17 ('1.31027747574118', '103.795371163103')
18 ('1.31067976097165', '103.795260908624')
19 ('1.35664284566921', '103.889828868222')
20 ('1.37248308734511', '103.893788141349')
21 ('1.33487526118006', '103.746894773008')
22 ('1.33305999269581', '103.743503708169')
23 ('1.31461849889242', '103.894738004768')
24 ('1.30192883841134', '103.876208170362')
25 ('1.3850

In [41]:
malls_coordinates = pd.DataFrame({'name': mall_names, 'latitude': latitude_list, 'longitude': longitude_list})
malls_coordinates

,name,latitude,longitude
0,100 AM,1.27458821795426,103.84347073661
1,600 @ Toa Payoh,1.33403623894465,103.850977706475
2,Anchorpoint,1.28861469030586,103.805009964592
3,Beauty World Centre,1.34241264188642,103.776539385406
4,Beauty World Plaza,1.34180018619223,103.776259359854
...,...,...,...
169,Scotts Shopping Centre,NA,NA
170,Serangoon Plaza,NA,NA
171,Shaw Tower,1.29664455421758,103.856855656294
172,Specialists' Shopping Centre,NA,NA


In [45]:
count = (malls_coordinates['latitude'] == 'NA').sum()
count


17

Unable to find the coordinates of 17 malls via the API. They either do not exist or have been closed down.

In [51]:
malls_coordinates_missing = malls_coordinates[malls_coordinates['latitude'] == 'NA']
malls_coordinates_missing

,name,latitude,longitude
7,Change Alley,NA,NA
9,City Vibe,NA,NA
62,Clarke Quay Central,NA,NA
66,Cosford Container Park,NA,NA
87,Mandarin Gallery,NA,NA
106,Shaw House and Centre,NA,NA
158,Amber Mansions,NA,NA
160,Capitol Centre,NA,NA
161,Ellenborough Market,NA,NA
163,Fitzpatrick's,NA,NA


Of the above, the followings malls have been closed:
- The Verge
- City Vibe
- JCube
- Jurong Entertainment Centre
- Ellenborough Market
- Capitol Centre
- Amber Mansions
- Serangoon Plaza
- Specialist Shopping Centre

The coordinates of the following malls are manually added:
- Clarke Quay Central
- Scotts Shopping Centre -> Scotts Square
- Shaw House and Centre -> Shaw House
- Mandarin Gallery
- Cosford Container Park
- Change Alley


In [ ]:
# read csv of manual list

In [ ]:
# add opening and closing dates